# 12

In [50]:
import pandas as pd
import math
import numpy as np
from numpy.linalg import inv
pd.options.mode.chained_assignment = None 

In [119]:
train=pd.read_csv('hw2_adaboost_train.dat.txt',sep=' ',header=None)
test=pd.read_csv('hw2_adaboost_test.dat.txt',sep=' ',header=None)
train.columns =['x1', 'x2', 'y']
test.columns =['x1', 'x2', 'y']

In [150]:
def cost(train):
    return sum(train['not_equal']*train['u'])/sum(train['u'])

def decision_stump(dim,s,train,sita):
    train['not_equal']=0
    train['y_hat'][train[dim] < temp_sita]=-1*s
    train['y_hat'][train[dim] >= temp_sita]=1*s
    train['not_equal'][train['y_hat']!=train['y']]=1
    
def sign(num):
    if(num>0):
        return 1
    else:
        return -1

In [169]:
cols=['x1','x2']
G = {'alpha':[],'s':[],'Ein':[],'dimension':[],'sita':[],'u':[]}
train['y_hat']=0
train['u']=[1/len(train)]*len(train)
n=0
while(n<300):
    Ein=float('inf')
    for i in cols:
        train=train.sort_values(by=[i])
        for j in range(len(train)-1):
            
            if(j==0):
                temp_sita=float('-inf')
            else:
                temp_sita=(train[i].iloc[j]+train[i].iloc[j+1])/2
                
            decision_stump(i,1,train,temp_sita)

            temp_Ein1=cost(train)
            
            decision_stump(i,-1,train,temp_sita)

            temp_Ein2=cost(train)
            

            if(temp_Ein1<temp_Ein2):
                if(Ein>temp_Ein1):
                    Dimension=i
                    Ein=temp_Ein1
                    s=1
                    sita=temp_sita
            else:
                if(Ein>temp_Ein2):
                    Dimension=i
                    Ein=temp_Ein2
                    s=-1
                    sita=temp_sita
         
              
                    
    #calculate new u
    if(Ein!=0):
        t=((1-Ein)/Ein)**0.5
    else:
        t=float('inf')
    
    G['u'].append(sum(train['u']))
    train['y_hat'][train[Dimension]>=sita]=s*1
    train['y_hat'][train[Dimension]<sita]=s*-1
        
    train['u'][train['y_hat']==train['y']]=train['u'][train['y_hat']==train['y']]/t    
    train['u'][train['y_hat']!=train['y']]=train['u'][train['y_hat']!=train['y']]*t
        
        
    #record g()
    G['alpha'].append(math.log(t))
    G['Ein'].append(Ein)
    G['s'].append(s)
    G['dimension'].append(Dimension)
    G['sita'].append(sita)
        
    n+=1

In [170]:
G=pd.DataFrame(G)

In [171]:
G

,alpha,s,Ein,dimension,sita,u
0,0.576340,1,0.240000,x1,0.644854,1.000000
1,0.762494,-1,0.178728,x2,0.626233,0.854166
2,0.588619,-1,0.235549,x2,0.232890,0.654504
3,0.538151,1,0.254206,x1,0.224440,0.555467
4,0.578833,1,0.239092,x1,0.876278,0.483716
...,...,...,...,...,...,...
295,0.176013,1,0.412891,x1,0.876278,0.005729
296,0.150156,-1,0.425481,x1,0.847283,0.005641
297,0.134103,1,0.433348,x1,0.542874,0.005578
298,0.152132,-1,0.424516,x2,0.374128,0.005528


In [175]:
train=train.sort_index()

In [176]:
for i in range(len(G)):
    train[i]=0
    dim=G['dimension'].iloc[i]
    sita=G['sita'].iloc[i]
    alpha=G['alpha'].iloc[i]
    s=G['s'].iloc[i]
    
    train[i][train[dim]>=sita]=s*1*alpha
    train[i][train[dim]<sita]=s*-1*alpha

res=train[range(0,299)]


In [177]:
y_hat=[]
train=train[['x1','x2','y']]
for i in range(len(res)):
    y_hat.append(sign(sum(res.iloc[i])))

train['y_hat']=y_hat

In [178]:
len(train[train['y']!=train['y_hat']])

0

# 12

In [174]:
print(G['Ein'].iloc[0])

0.2399999999999999


# 13

In [179]:
len(train[train['y']!=train['y_hat']])

0

# 14

In [183]:
G['u'].iloc[2]

0.6545039637744694

# 15

In [184]:
G['u'].iloc[299]

0.005465124119345906

# 16

In [187]:
min(G['Ein'])

0.1787280701754384

# 17

In [189]:
i=1
test[i]=0
dim=G['dimension'].iloc[i]
sita=G['sita'].iloc[i]
s=G['s'].iloc[i]

test[i][test[dim]>=sita]=s*1
test[i][test[dim]<sita]=s*-1

print(len(test[test[i]!=test['y']])/len(test))



0.262


# 18

In [190]:
for i in range(len(G)):
    test[i]=0
    dim=G['dimension'].iloc[i]
    sita=G['sita'].iloc[i]
    alpha=G['alpha'].iloc[i]
    s=G['s'].iloc[i]
    
    test[i][test[dim]>=sita]=s*1*alpha
    test[i][test[dim]<sita]=s*-1*alpha

res=test[range(0,299)]

y_hat=[]
test=test[['x1','x2','y']]
for i in range(len(res)):
    y_hat.append(sign(sum(res.iloc[i])))

test['y_hat']=y_hat

print(len(test[test['y']!=test['y_hat']])/len(test))

0.127


# 19

In [225]:
df=pd.read_csv('hw2_lssvm_all.dat.txt',sep=' ',header=None)
df=df[range(1,12)]

In [296]:
#Train Test split
train=df.iloc[range(0,401)]
test=df.iloc[range(401,500)]

In [297]:
x_test=test[range(1,11)]
x_test[0]=1
y_test=test[11]
x_test=x_test.to_numpy()
x_test=np.asmatrix(x_test)

In [298]:
x_train=train[range(1,11)]
x_train[0]=1
x_train=x_train.to_numpy()
x_train=np.asmatrix(x_train)
y_train=train[11]

In [229]:
gamma=[32, 2, 0.125]
Lambda=[0.001, 1, 1000]

In [285]:
def turn_to_power(list, power=2): 
    return [number**power for number in list]

def kernal_Gaussian_RBF(X,gamma):
    m, n = X.shape
    K = np.zeros((m,m))
    for i in range(m):
        for j in range(m):
            xi=X[i,:]
            xj=X[j,:]
            delta=xi-xj
            delta=turn_to_power(delta.tolist()[0])
            K[i,j]=math.exp(-gamma*np.sum(delta))
    return K

def beta(Lambda,K,Y):
    m, n = K.shape
    I=np.identity(m)
    return inv((Lambda*I+K)).dot(Y)

def predict(gamma,Beta,xn,x):
    delta=np.array(x-xn)
    kernel=np.exp(-gamma*np.sum(delta**2,axis=1))
    return kernel.dot(Beta)

def sign(x):
    if(x>0):
        return 1
    else:
        return -1

def error_in(gamma,Beta,X,Y):
    
    y_predict=[]
    for i in range(len(X)):
          y_predict.append(sign(predict(gamma,Beta,X[i],X)))
    
    return len(Y[y_predict!=Y])/len(Y)


def error_out(gamma,Beta,X,X_test,Y):
    
    y_predict=[]
    for i in range(len(X_test)):
          y_predict.append(sign(predict(gamma,Beta,X_test[i],X)))
    
    return len(Y[y_predict!=Y])/len(Y)
    

In [299]:
X=x_train
X_test=x_test
Y=y_test

In [294]:
X_test

,1,2,3,4,5,6,7,8,9,10,0
401,6.315,5.439,-6.615,-8.930,2.978,0.236,0.837,3.567,0.473,-5.990,1
402,5.748,4.537,4.808,-7.896,-4.014,8.878,-5.360,-4.223,2.137,-10.975,1
403,2.815,3.914,0.924,5.112,-6.903,-3.916,-2.724,-6.768,4.858,6.134,1
404,-3.186,1.439,-0.112,8.496,5.587,0.599,1.068,-7.244,-0.673,-7.200,1
405,-2.985,0.269,-7.147,-7.062,8.975,0.642,-4.198,4.277,-2.460,-6.809,1
...,...,...,...,...,...,...,...,...,...,...,...
495,-4.915,-6.687,-2.742,7.010,1.958,4.891,-4.081,2.878,-0.622,-10.494,1
496,5.312,3.225,-7.290,-10.544,1.038,1.133,0.454,3.726,2.000,-5.998,1
497,-5.886,2.038,-1.136,7.281,5.380,-2.020,1.388,-7.267,-2.814,-8.637,1
498,-4.278,-6.023,-4.698,7.758,0.745,3.107,-3.645,3.498,1.437,-9.395,1


In [302]:
for i in range(len(gamma)):
    for j in range(len(Lambda)):
        K=kernal_Gaussian_RBF(x_train,gamma[i])
        Beta=beta(Lambda[j],K,y_train)
        Ein=error_in(gamma[i],Beta,x_train,y_train)
        Eout=error_out(gamma[i],Beta,x_train,x_test,y_test)
        print(f'Gamma:{gamma[i]}')
        print(f'Lambda:{Lambda[j]}')
        print(f'Ein:{Ein}')
        print(f'Eout:{Eout}')
        print('-----------------')
        
        

Gamma:32
Lambda:0.001
Ein:0.0
Eout:0.4444444444444444
-----------------
Gamma:32
Lambda:1
Ein:0.0
Eout:0.4444444444444444
-----------------
Gamma:32
Lambda:1000
Ein:0.0
Eout:0.4444444444444444
-----------------
Gamma:2
Lambda:0.001
Ein:0.0
Eout:0.43434343434343436
-----------------
Gamma:2
Lambda:1
Ein:0.0
Eout:0.43434343434343436
-----------------
Gamma:2
Lambda:1000
Ein:0.0
Eout:0.43434343434343436
-----------------
Gamma:0.125
Lambda:0.001
Ein:0.0
Eout:0.46464646464646464
-----------------
Gamma:0.125
Lambda:1
Ein:0.032418952618453865
Eout:0.45454545454545453
-----------------
Gamma:0.125
Lambda:1000
Ein:0.24189526184538654
Eout:0.3939393939393939
-----------------


In [304]:
print('Error in is between 0 - 0.24')

Error in is between 0 - 0.24


# 20

In [305]:
print("minimum error out is Eout:0.3939393939393939")

minimum error out is Eout:0.3939393939393939
